Task 1

In [ ]:
import json
import requests

WIKIDATA_API_ENDPOINT = "https://www.wikidata.org/w/api.php?"
WIKIPEDIA_API_ENDPOINT = "https://en.wikipedia.org/w/api.php"
HEADERS = {"User-Agent": "coursework_query/2.0"}

In [ ]:
def get_turing_award_recipients():
    acm_award_entities = []
    
    params_query = {
        "action":"query",
        "format":"json",
        "formatversion":"latest",
        "list":"search",
        "srsearch": "haswbstatement:P166=Q185667",
        "srlimit":100
    }
    
    search_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=params_query)
    data = search_response.json()
    for result in data['query']['search']:
        acm_award_entities.append(result['title'])
    return acm_award_entities

get_turing_award_recipients()

Task 2

In [ ]:
def get_wikipedia_content(entity_id):
    
    params_wbgetentities = {
		"action": "wbgetentities",
		"format": "json",
		"ids": entity_id,
		"sites": "",
		"props": "sitelinks/urls",
		"languages": "en",
		"sitefilter": "enwiki",
		"utf8": 1,
		"ascii": 1,
		"formatversion": "latest"
	}
    
    wbgetentities_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=params_wbgetentities)
    wbgetentities_data = wbgetentities_response.json()
    recipient_name = wbgetentities_data['entities'][entity_id]['sitelinks']['enwiki']['url'].split("https://en.wikipedia.org/wiki/")[1]
    
    params_getcontent={
        'action': 'query',
		'format': 'json',
		'titles': recipient_name,
		'prop': 'extracts',
		'explaintext': True,
	}
    
    wbgetentities_response = requests.get(WIKIPEDIA_API_ENDPOINT, headers=HEADERS, params=params_getcontent)
    wbgetentities_data = wbgetentities_response.json()
    content = next(iter(wbgetentities_data['query']['pages'].values()))['extract']
    return content
    
print(get_wikipedia_content("Q92758"))
